# 0. Intro

The aim of this "document" is to go trough the steps needed to segment the grains (~10-100nm in diametar) on the images from the AFM.<br>
<br>
We use Bruker AFM for scanning the surface and all the data treated here comes from this microscope. <br><br>
I'll divide the problem into four parts (for now) :
- __loading the data__ _(and extracting the parts that interest us)_
- __pretreatement__ _(denoising, edge enhancements)_
- __segmentation__ _(findig peaks + watershed)_
- __statistics__ _(to do...)_

#### 0.0.1. importing the necessary packages

In [1]:
%matplotlib widget
from ipywidgets import widgets
import numpy as np
# import pySPM
from skimage import io, exposure, img_as_float, filters, restoration, img_as_ubyte, morphology, img_as_uint, measure, segmentation
from skimage.morphology import disk
from tkinter import filedialog, Tk, messagebox
from matplotlib import pyplot as plt
from scipy import ndimage
import cv2 as cv
from warnings import warn

# I'll just use this cell for all the imports I will be needing later


params = {'phase_image_denoise_footprint':np.ones((5,3)), 
          'cv_bilateral_filter_spatial_reach':9, 
          'cv_bilateral_filter_color_reach':70, 
          'gaussian_blurr_small':3, 
          'gaussian_blurr_large':7,
          'n_iter_closing_edges':3,
          'weight_height_edges':5, 
          'weight_phase_edges':3,
          'weight_original_height_image':20, 
          'n_iter_opening_sharpened_img':2, 
          'peaks_min_distance':9, 
          'min_height_extrema':0.37, 
          'watershed_compactness_param':0.9994}



selem = morphology.disk(3)

# 1. Loading Data

There are several possiblities you can use to load your data (results from AFM Bruker @ svi):
- loading the image file that was (a bit) pretreated by the nanoscope software (native to Brucker (?))
- loading raw .spm (or similar extension) file directly produced by the microscope - __This is the approach to be priviledged in the future__
 * _one will need to use package like pySPM to be able to load the data in your python program</br> (it proved to be somewhat a hussle to install this package (anaconda installation got messsed-up), so go with it carefully)_
- use some other software (like ImageJ/fiji or Gwyddion) to open and/or pretreat the data/image and then save it to format easilly read by python (like .npy (not available) or hdf5!?!)

<font color=blue>**Note:**  It might be worth setting some rules on what to record and what parameters to use when doing AFM scans. Check with Hérvé et Barbara.</font></br>
It could help standardise the treatements. Anywhay, one good habit to 

## 1.1. ...from _.jpg_ images
(produced by nanoscope - the Brucker's software)

In [2]:
data_folder = '/home/dejan/Documents/python_learning/training/Data/ZnO/'
intermediate_results_folder = '/home/dejan/Documents/python_learning/training/Intermediate_results/'
prompt_for_images = 'Choose both height and phase image :'


# This is where you chooose the images to treat:

root = Tk()
root.withdraw()

file_path = filedialog.askopenfilenames(initialdir=data_folder, title=prompt_for_images)

if len(file_path) != 2:
    raise IOError(f'you chose {len(file_path)} file(s) instead of two')

name_height = file_path[0].split('/')[-1]
name_phase = file_path[1].split('/')[-1]

print(file_path[0])

if 'phase' not in name_phase[-10:-4]:
    warn(f'\nThe second image contains the string "{name_phase[-10:-4]}", where something like "phase" is expected\nYou should double-check your input images')


imh = io.imread(file_path[0], as_gray=True)
im1 = io.imread(file_path[1], as_gray=True)

# This next step had to be added to cope with height and phase images of the different sizes (?). It simply crops the bigger image to the size of the smaller one
if im1.shape == imh.shape:
    pass
else:
    common_shape = (min(imh.shape[0], im1.shape[0]),min(imh.shape[1], im1.shape[1]))
    im1 = im1[0:common_shape[0], 0:common_shape[1]]
    imh = imh[0:common_shape[0], 0:common_shape[1]]

/home/dejan/Documents/python_learning/training/Data/ZnO/E938_NICr_1_ZnOc_100nm_550deg_1h_vide_500n_heigh.jpg


It should be noted that the images are loaded with dtype=float64. Nevertheless, they only contain positive values, so there would be no significant loss (due to negative values clipping) when converting them to unisigned integers like uint8

plotting...

In [ ]:
print('----> The imh image is of shape: {0}, datatype {1}, max: {2}, min: {3}'.format(imh.shape, imh.dtype, np.amax(imh), np.amin(imh)))
print('----> The im1 image is of shape: {0}, datatype {1}, max: {2}, min: {3}'.format(im1.shape, im1.dtype, np.amax(im1), np.amin(im1)))

# Plotting the images...

fig, ax = plt.subplots(ncols=2, nrows=1, sharex=True, sharey=True, frameon=False, figsize=(16, 6))
ax[0].imshow(imh, cmap='gray')
ax[0].set_title("height image")
ax[0].axis('off') # to remove the window frame around the image

ax[1].imshow(im1, cmap='gray')
ax[1].set_title("phase image")
ax[1].axis('off')
plt.xticks([]), plt.yticks([])  # to hide tick values on X and Y axis
plt.show()

## 1.2. ...from _Brucker raw files (.spm)_

In order to read the .spm files, you need to install pySPM package. Just download the whole .zip from GitHub repo, unpack it to the location you want, than open conda prompt in that location and do `python setup.py install` (if  i'm not mistaken).<br><br>
Then, you can load the Bruker file via the pySPM python module:
[Notebook to read the Bruker raw .spc files](./Read_Bruker.ipynb)

## 1.3. ...from previously reformatted data

# 2. Pretreatement

## 2.1. Extracting the data
Depending on your input method, you would need to crop the images, or extract the data in some other way<br><br>
**Note:** _information about the length scale is definitly useful, so there should be a way to grab it and store somewhere. The height scale is not really used in this particular problem, but it could be potentially useful to know this ass well (only the heights span is needed as the further transformations of the image would mess up the scale anyway)_

### 2.1.1. Cropping nanoscope image
The images imported from nanoscope have the frame containing infos on scales (color and lengths), which we will not need in the tratements

In [215]:
def crop_afm_bar(im):
    """
    Crop image to remove blank borders and colorbar. Are the dimensions
    always the same? To be checked
    """
    return im[22:-56, 4:-50]

img0 = crop_afm_bar(imh)
imph0 = crop_afm_bar(im1)

### 2.1.2. Choosing the channels from Bruker .spm files

### 2.1.3. Loading the image files saved by other programs

Should be straightforward, depends on the export you've made

### - plotting the images and histograms and checking the data formats

In [ ]:
print('----> The img image is of shape: {0}, datatype {1}, max: {2}, min: {3}'.format(img.shape, img.dtype, np.amax(img), np.amin(img)))
print('----> The imph image is of shape: {0}, datatype {1}, max: {2}, min: {3}'.format(imph.shape, imph.dtype, np.amax(imph), np.amin(imph)))


fig, ax = plt.subplots(2,2, figsize =(12,8))

ax[0,0].imshow(img0, cmap='gray')
ax[0,0].set_title("height image")
ax[1,0].hist(img0.ravel(), bins=50)
# ax[1,0].set_title("height image histogram")
ax[0,1].imshow(imph0, cmap='gray')
ax[0,1].set_title("phase image")
ax[1,1].hist(imph0.ravel(), bins=50)

plt.show()


in most cases, we would probably like to equilaze histograms.

### - Saving the pretreated images in standardized format for easier reloading

**Note:** _What format to use? For starters, .npy, but should switch to something more descriptive and standardized. hdf5 might be a valid candidate ?!? Would it be worthwhile to create the database of thus standardized data? I could start by creating my own database. Should be carefull on choosing enough metadata (a lot of it is contained already in the headers of Bruker raw files)_

In [ ]:
# Save croped images as numpy binary files (faster to load?):
np.save(intermediate_results_folder+'croped_'+name_height[:-4], img0)
np.save(intermediate_results_folder+'croped_'+name_phase[:-4], imph0)

himg_filename = intermediate_results_folder+'cropped_'+name_height
phimg_filename = intermediate_results_folder+'cropped_'+name_phase
%store himg_filename

# Or, save the same cropped images as .jpg/.png files (smaller space on disk):
io.imsave(himg_filename, img0)
io.imsave(phimg_filename, imph0)
# Attention, saving files to the .png/.jpg format (uint16/uint8) from float64 used when the data was loaded sets off the warning about presision loss

## 2.2. Histogram equalization and Denoising

We have a choice of equilizing the histograms locally or globally. On the denoising side, median filter or/and bilateral filter should be used.

### 2.2.1. Global histogram equalization 

In [216]:
img=exposure.equalize_hist(img0)
imph=exposure.equalize_hist(imph0)

plotting...

In [ ]:
print('----> The img image is of shape: {0}, datatype {1}, max: {2}, min: {3}'.format(img.shape, img.dtype, np.amax(img), np.amin(img)))
print('----> The imph image is of shape: {0}, datatype {1}, max: {2}, min: {3}'.format(imph.shape, imph.dtype, np.amax(imph), np.amin(imph)))

ig, ax = plt.subplots(2,2, figsize =(12,8))

ax[0,0].imshow(img, cmap='gray')
ax[0,0].set_title("height image")
ax[1,0].hist(img.ravel(), bins=50)
# ax[1,0].set_title("height image histogram")
ax[0,1].imshow(imph, cmap='gray')
ax[0,1].set_title("phase image")
ax[1,1].hist(imph.ravel(), bins=50)

plt.show()

### 2.2.2. Local histogram equalization

In [217]:
imh_u8 = img_as_ubyte(img0)
im1_u8 = img_as_ubyte(imph0)
clahe = cv.createCLAHE()
imh_u8 = clahe.apply(imh_u8)
im1_u8 = clahe.apply(im1_u8)

In [ ]:
print('----> The imh with equalized histogram is of shape: {0}, datatype {1}, max: {2}, min: {3}'.format(imh_u8.shape, imh_u8.dtype, np.amax(imh_u8), np.amin(imh_u8)))
print('----> The im1 with equalized histogram is of shape: {0}, datatype {1}, max: {2}, min: {3}'.format(im1_u8.shape, im1_u8.dtype, np.amax(im1_u8), np.amin(im1_u8)))


fig, ax = plt.subplots(2,2, figsize =(12,8))
ax[0,0].imshow(imh_u8, cmap='gray')
ax[0,1].imshow(im1_u8, cmap='gray')
ax[1,0].hist(imh_u8.ravel(), bins=50)
#ax[0,1].imshow(img, cmap='gray')
ax[1,1].hist(im1_u8.ravel(), bins=50)

plt.show()

### 2.2.3. Denoising...using median_filter

In [218]:
# I only choose to run the median filter on the phase image, to try to remove the horizontal lines from scans

im1_u8 = ndimage.median_filter(im1_u8, footprint = params['phase_image_denoise_footprint'])

In [110]:
fig, ax = plt.subplots(1,2, figsize =(12,8))
ax[0].imshow(im1_u8, cmap='gray')
ax[1].imshow(imph0, cmap='gray')
plt.show()

FigureCanvasNbAgg()

### 2.2.4. Denoising ...using bilateral filter

In [111]:
# import skimage.restoration
imh_denoised_bilateral = restoration.denoise_bilateral(imh_u8, sigma_color=0.2, sigma_spatial=8, multichannel=False)

In [219]:
# import cv2 as cv
img_cdb = cv.bilateralFilter(imh_u8, params['cv_bilateral_filter_spatial_reach'], params['cv_bilateral_filter_color_reach'], params['cv_bilateral_filter_color_reach'])
imph_cdb = cv.bilateralFilter(im1_u8,params['cv_bilateral_filter_spatial_reach'], params['cv_bilateral_filter_color_reach'], params['cv_bilateral_filter_color_reach'])

plotting...

In [113]:
# Plot the resulting images
fig, ax = plt.subplots(ncols=2, nrows=2, sharex=True, sharey=True, frameon=False, figsize=(8, 6))
ax[0,0].imshow(img0, cmap='gray')
ax[0,0].set_title("height image")
ax[0,0].axis('off') # to remove the window frame around the image

ax[0,1].imshow(imph0, cmap='gray')
ax[0,1].set_title("phase image")
ax[0,1].axis('off')

ax[1,0].imshow(img_cdb, cmap='gray')
ax[1,0].set_title("img_cdb")
ax[1,0].axis('off') # to remove the window frame around the image

ax[1,1].imshow(imph_cdb, cmap='gray')
ax[1,1].set_title("imph_cdb")
ax[1,1].axis('off')
plt.xticks([]), plt.yticks([])  # to hide tick values on X and Y axis

plt.tight_layout()
plt.show()

FigureCanvasNbAgg()

## 2.3. Enhancing the edges / Sharpening

### 2.3.1. ...using the difference of Gaussians

Here, we try to accentuate the borders by first blurring the image a bit, then substracting the blurred image from the original one which should stress out the edges

In [220]:
def find_edges(img, small_blur = params['gaussian_blurr_small'], large_blur = params['gaussian_blurr_large'], it_max=params['n_iter_closing_edges'], selem=selem):
    blurred_small = ndimage.gaussian_filter(img, small_blur)
    blurred_large = ndimage.gaussian_filter(img, large_blur)
    edges = ~img_as_uint(blurred_large - blurred_small)
    edges = clahe.apply(edges)
    it=0
    while it<it_max:
        edges = morphology.closing(edges, selem=selem)
        it+=1
    edges = 255 * (edges / edges.max())
    return(edges)

edges = find_edges(img_cdb)
edges_phase = find_edges(img_cdb)

### 2.2.2. ...or using *skimage.enhance_contrast*

In [138]:
# from skimage import filters
# from skimage.morphology import disk

imh_sharpened_eh = filters.rank.enhance_contrast(img_cdb, disk(2))
im1_sharpened_eh = filters.rank.enhance_contrast(imph_cdb, disk(2))


plotting...

In [ ]:

# Plot the resulting images
fig, ax = plt.subplots(ncols=2, nrows=2, sharex=True, sharey=True, frameon=False, figsize=(12, 8))
ax[0,0].imshow(img_cdb, cmap='gray')
ax[0,0].set_title("height image denoised")
ax[0,0].axis('off') # to remove the window frame around the image

ax[0,1].imshow(imph_cdb, cmap='gray')
ax[0,1].set_title("phase image denoised")
ax[0,1].axis('off')

ax[1,0].imshow(edges, cmap='gray')
ax[1,0].set_title("edges on height denoised image")
ax[1,0].axis('off') # to remove the window frame around the image

ax[1,1].imshow(edges_phase, cmap='gray')
ax[1,1].set_title("edges on phase denoised image")
ax[1,1].axis('off')
plt.xticks([]), plt.yticks([])  # to hide tick values on X and Y axis

plt.tight_layout()
plt.show()

## 2.4. Constructing the combined image wich will be the base for the rest

In [221]:

# enhancing the edges on the denoised height image by substracting the difference of gaussians from both phase and height images (wighted: 

koka = img_as_uint(params['weight_original_height_image']*np.asarray(img_cdb, dtype=np.int32) - params['weight_height_edges']*np.asarray(edges, dtype=np.int32) - params['weight_phase_edges']*np.asarray(edges_phase, dtype=np.int32))
# img_as_uint clips the negative values to zero
#print(f'min koka = {koka.min()}, max koka = {koka.max()}')


sharp_gauss = 255 * (koka / koka.max()) # this is the image that we will use to find the grains (uint8)


# we should bare in mind that the size of the grains is rather enlarged on the AFM images,
# as a result of the scanning probe size, so opening a bit won't hurt, and might permit resolving the grains better



/home/dejan/anaconda3/envs/segmentation/lib/python3.7/site-packages/skimage/util/dtype.py:190: UserWarning: Downcasting int32 to uint16 without scaling because max value 5084 fits in uint16
  "value {} fits in {}".format(a.dtype, dtype, a.max(), dtype))


# 3. Marking the grains

## 3.1. Creating masks...
  from the combined image

In [222]:
#----------------------Testing different masks-----------------------------------
win_size = 61
koef_sigma = 0.003

thresh_loc = filters.threshold_local(sharp_gauss, win_size, method='gaussian', offset=0.1, mode='reflect', param=7)
thresh_li = filters.threshold_li(sharp_gauss)
thresh_nib = filters.threshold_niblack(sharp_gauss, window_size=win_size, k=koef_sigma)
thresh_sau = filters.threshold_sauvola(sharp_gauss, window_size=win_size, k=koef_sigma, r=None)
thresh_otsu = filters.threshold_otsu(sharp_gauss, nbins=256)

mask_li = sharp_gauss > thresh_li
mask_nib = sharp_gauss > thresh_nib
mask_sau = sharp_gauss > thresh_sau
mask_otsu = sharp_gauss > thresh_otsu
mask_loc = sharp_gauss > thresh_loc

mask = mask_nib # settling with one of the above
morphology.remove_small_holes(mask, 96, connectivity=1, in_place=True)
# =============================================================================
#     There is a problem with local threshold filter:
#     In the areas where is a high-level plateau, using this threshold,
#     There are some blanks obtained even though it is clearly not the background
#     So after all, back to the global threshold
# =============================================================================



it=0
while it<params['n_iter_opening_sharpened_img']:
    mask = morphology.opening(mask, selem = selem)
    it+=1
    
 

## 3.2. Creating the distance map on the combined image...

In [223]:
# getting the map of the distances from the background on the mask:
distance_map = ndimage.distance_transform_edt(mask)
#distance_map = exposure.rescale_intensity(distance_map)


## 3.3. Finding peaks

In [224]:
footprint = morphology.disk(9)
def find_peaks_ongrains(img, footprint=footprint, labels=mask, connectivity=2, selem=selem):
    peaks = feature.peak_local_max(img, footprint=footprint, labels=mask, indices=False)
    peaks = morphology.binary_dilation(peaks, selem=selem)
    peak_labels = measure.label(peaks)
    peak_indices = feature.peak_local_max(img, footprint=footprint, labels=mask, indices=True)
    return peaks, peak_labels, peak_indices


#------------------Finding  peaks using the distance map--------------------------------
# peaks are then set as the points the furthest away (spatially, not by intenisty) from the background 
#(this might be especially convinient for some images where the brightest spots are close to the grain edges)

peaks, peak_labels, peak_indices = find_peaks_ongrains(distance_map)


#-------------- Finding peaks from sharpened image (diff of gauss)-------------
#---------------- based on heights (grayscale values) -------------------------

peaks_height, peaks_height_labels, peaks_height_indices = find_peaks_ongrains(sharp_gauss)

%store peaks_height_indices

Stored 'peaks_height_indices' (ndarray)


## 3.4. Labelizig the grains

In [225]:
#----------------------------------------------------------------------------------

# watershedding:

def find_grains_onpeaks(peak_labels, img=-sharp_gauss, compactness=params['watershed_compactness_param'], watershed_line=True, mask=mask):
    _grain_labels = morphology.watershed(img, peak_labels, compactness = compactness, mask=mask, watershed_line=watershed_line)
    _grain_labels = segmentation.clear_border(_grain_labels)
    new_order = np.random.permutation(np.arange(101, _grain_labels.max()+102))
    new_labels = new_order[_grain_labels]
    new_labels[mask==0]=0
    new_labels = segmentation.clear_border(new_labels)
    return new_labels
    

grains = find_grains_onpeaks(peak_labels) # from the distance map
grains_height = find_grains_onpeaks(peaks_height_labels) # from sharp_gauss height values



# 4. Plotting...

In [226]:
alpha_grains = 0.22
alpha_peaks = 0.1
@widgets.interact(alpha_grains=(0.0,0.6))
def plot_ilustration(alpha_grains):
    plt.close('all')
    fig, ax = plt.subplots(ncols=3, nrows=2, sharex=True, sharey=True, frameon=False, figsize=(12, 8))

    ax[0,0].imshow(img, cmap='gray')
    ax[0,0].set_title("original height image")

    ax[0,1].imshow(sharp_gauss)
    ax[0,1].set_title(f"sharp_gauss \n combined image used for grain detection")#win size={win_size}, k={koef_sigma}")

    ax[0,2].imshow(imph, cmap='gray')
    ax[0,2].set_title("original phase image")


    ax[1,0].imshow(img, cmap='gray')
    ax[1,0].imshow(grains, cmap='tab20b', alpha=alpha_grains)
    ax[1,0].contour(peaks, colors='green', alpha=alpha_peaks)
    ax[1,0].set_title("grains traced on top of the original height image\n seed: find_loc_max on mask distance map")

    #imshow(grains_height, cmap='nipy_spectral')

    ax[1,1].imshow(mask, cmap='gray')
    ax[1,1].set_title("mask")#distance map used to find peaks on the left")


    ax[1,2].imshow(img, cmap='gray')
    ax[1,2].contour(peaks_height, colors='yellow', alpha=alpha_peaks)
    ax[1,2].imshow(grains_height, cmap='tab20b', alpha=alpha_grains)
    ax[1,2].set_title("grains traced on top of the original height image\n seed: find_loc_max on sharp_gauss")

    axis = ax.ravel()
    for a in axis:
        a.axis('off') # to remove the window frame around the image

    #plt.xticks([]), plt.yticks([])  # to hide tick values on X and Y axis
    plt.tight_layout()
    plt.show()                                                                                  


interactive(children=(FloatSlider(value=0.3, description='alpha_grains', max=0.6), Output()), _dom_classes=('w…

# 5. Extras

In [144]:
@widgets.interact(win_size=widgets.IntSlider(min=9,max=131, step=2, value=31), koef_sigma=(-0.95,0.95))
def alh_ph(win_size,koef_sigma):
    plt.close('all')
    thresh_nib = filters.threshold_niblack(sharp_gauss, window_size=win_size, k=koef_sigma)
    mask_nib = sharp_gauss > thresh_nib
    plt.figure(figsize=(12,6))
    #plt.imshow(kanali, cmap='gray')
    plt.imshow(mask_nib, cmap='gray')
    #plt.imshow(mmm, cmap='spring', alpha = a)
    plt.show()

interactive(children=(IntSlider(value=31, description='win_size', max=131, min=9, step=2), FloatSlider(value=0…

In [97]:
from IPython.display import display
from bqplot import pyplot as bqp

x_data = peaks_height_indices[:,0]
y_data = peaks_height_indices[:,1]


bqp.figure(title='Illustration of peak reajustement')#, background_style = {'background':'url('+himg_filename+')', 'background-size': 'contain', 'opacity':0.5}) # {'background_image': height_image})
#slika = bqp.imshow(height_image, 'height')
scatter_plot = bqp.scatter(x_data, y_data)
bqp.show()

output = widgets.Output() 
@output.capture()
def foo(change):
    #print(f'the y_data has changed from {change.old} to {change.new}')
    print(change.new)

scatter_plot.observe(foo, 'y')
scatter_plot.enable_move = True

In [99]:
#display(output)